Stable version:

In [ ]:
"""
NZTA envelope-first optimiser – COPT (Fixed Funding + Piecewise Soft Cap) v94.14

Hybrid of v94.11 (stable/fast defaults) and v94.13 (multi‑phase + Victory Lap).

Key design:
──────────────────────────────────
1. THREADS / PARAM DEFAULTS (back to 94.11 behaviour):
   - Force single-threaded BLAS / COPT by default for stability.
   - COPT model init:
       Threads     = 1
       MipStartMode= 2
       Presolve    = 1
       Scaling     = 1

2. PROFILE‑DEPENDENT SOLVE STRATEGY:

   (a) fast, balanced  → Single-phase (v94.11 style)
       - Just TimeLimit + RelGap
       - Optional precision top-up to tighten gap if we already have an incumbent.

   (b) thorough        → 2‑Phase (Probe + Verify + Victory Lap)
       - Phase 1: Probe (≈10% of budget). Quick warm-up.
       - Phase 2: Verify with dynamic Victory Lap:
           • Callback monitors BestObj/BestBnd.
           • When gap < target, start a 60s “Victory Lap” then stop.
           • If gap never hits target, stop at the normal time limit.

   (c) ultra           → 3‑Phase (Probe + Verify + Victory Lap + Grind)
       - Phase 1: Probe (10%).
       - Phase 2: Verify + Victory Lap (30%).
           • If gap < target and Victory Lap completes → skip Phase 3.
       - Phase 3: Grind with remaining time (up to full budget).

   All phases *only* manipulate TimeLimit / RelGap; no aggressive heuristics or
   exotic search controls – we let COPT do its job with the proven stable
   defaults from v94.11.

3. CALLBACK ROBUSTNESS:
   - Uses coptpy.CallbackBase + getInfo(COPT.CbInfo.BestObj/BestBnd) to compute
     gap inside the callback (no reliance on non-portable helpers).
   - Auto‑detects callback context constants (MIPSOL / MIPNODE); falls back
     gracefully to single‑phase solve if callbacks aren’t supported.

4. BUGFIXES vs earlier three‑phase code:
   - orchestrate_total no longer references non-existent res.best_solution_values.
   - Victory Lap callback now uses getInfo instead of getBestGap/hasIncumbent.
"""

from __future__ import annotations

import os

# ─────────────────────────────────────────────────────────────────────
# THREADING CONTROL
# ─────────────────────────────────────────────────────────────────────
# Default: mimic v94.11 (single-threaded BLAS & COPT) for stability.
FORCE_SINGLE_THREAD = False

if FORCE_SINGLE_THREAD:
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["OPENBLAS_NUM_THREADS"] = "1"
    os.environ["MKL_NUM_THREADS"] = "1"
    os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
    os.environ["NUMEXPR_NUM_THREADS"] = "1"
    # COPT respects both this env var and the 'Threads' model parameter.
    os.environ["COPT_NUM_THREADS"] = "1"

import sys
import time
import re
import math
import hashlib
import random
import pickle
import json
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Iterable, Any
from collections import defaultdict

import numpy as np
import pandas as pd

# --- COPT --------------------------------------------------------------------
try:
    import coptpy as co
except Exception as e:
    raise RuntimeError(
        "coptpy is required. Please ensure COPT is installed and licensed "
        "(pip install coptpy) and COPT_HOME is configured."
    ) from e

COPT = co.COPT

# --- CALLBACK CONSTANT DISCOVERY --------------------------------------------
def _get_copt_constant(names: List[str], default: int = 0) -> int:
    """
    Try multiple possible constant names to support different COPT versions.
    Returns 0 if none are found.
    """
    for name in names:
        if hasattr(COPT, name):
            return getattr(COPT, name)
    return default

# Typical MIP callback contexts (names differ slightly across versions)
CTX_MIPSOL = _get_copt_constant(["CBC_MIP_SOL", "CB_MIPSOL", "MIPSOL"], 0)
CTX_MIPNODE = _get_copt_constant(["CBC_MIP_NODE", "CB_MIPNODE", "MIPNODE"], 0)

if CTX_MIPSOL == 0 and CTX_MIPNODE == 0:
    print(
        "Warning: Could not detect COPT MIP callback context constants. "
        "Victory Lap callback may not be active."
    )

# ─────────────────────────────────────────────────────────────────────
# PATHS
# ─────────────────────────────────────────────────────────────────────
ROOT = Path(r"C:\Users\Adrian Desilvestro\Documents\NZTA\Project_Rons_optimisation")
DATA_FILE = ROOT / "Cost_benefit_streams.xlsx"
CACHE = ROOT / "scenario_cache_benefit_mo"
CACHE.mkdir(exist_ok=True)
PKL_PREFIX: str | None = "mathematical best_RONS_"

# ─────────────────────────────────────────────────────────────────────
# CALENDAR / ECON
# ─────────────────────────────────────────────────────────────────────
START_FY = 2026
FINAL_YEAR = 2095
TFIXED = int(FINAL_YEAR - START_FY + 1)
YEARS = TFIXED

BENEFIT_DISCOUNT_RATE = 0.02

# Scaling: 1 unit of spend in the model = 0.01 M (10k)
SPEND_SCALE = 100       # 1 unit = 0.01 M
PV_SCALE = 10000        # integer PV scaling

MAX_STARTS_PER_FY = 100  # Max project starts allowed in a single year

# ─────────────────────────────────────────────────────────────────────
# FUNDING ENVELOPE CONFIGURATION
# ─────────────────────────────────────────────────────────────────────
TAPER_YEARS = 0

# ─────────────────────────────────────────────────────────────────────
# PIECEWISE SOFT CAP SETTINGS (The "Tax Brackets")
# ─────────────────────────────────────────────────────────────────────
PIECEWISE_CAP_TIERS: List[Tuple[float, float]] = [
    (0.12, 1000.0),
    (0.15, 4000.0),
    (0.20, 12000.0),
]

# ─────────────────────────────────────────────────────────────────────
# OBJECTIVE WEIGHTS
# ─────────────────────────────────────────────────────────────────────
BACKLOG_WEIGHT = 1.0            # Penalise normal net balance
PV_WEIGHT = 1e-4                # PV tie-breaker

# ─────────────────────────────────────────────────────────────────────
# SOLVER GLOBALS
# ─────────────────────────────────────────────────────────────────────
# Match v94.11: one worker thread, deterministic seed.
SOLVER_THREADS_DEFAULT = 0
SOLVER_SEED_DEFAULT = 17
VERBOSE = 2

# Profile grid (time in seconds, target relative gap)
OPTIMISATION_PROFILE = "ultra"  # "fast", "balanced", "thorough", "ultra"
EFFORT: Dict[str, Dict[str, float]] = {
    "fast":     {"MO": 60.0,  "REL_GAP": 0.020},
    "balanced": {"MO": 120.0, "REL_GAP": 0.015},
    # thorough: more time, tighter gap
    "thorough": {"MO": 360.0, "REL_GAP": 0.003},  # 6m, 0.3%
    # ultra: maximum rigour
    "ultra":    {"MO": 900.0, "REL_GAP": 0.001},  # 15m, 0.1%
}

# Legacy tuning knobs from v94.11 (used in single‑phase solver)
FAST_STOP_GAP = 0.007
CONVERGENCE_REPEAT_N = 2
PRECISION_TOPUP_TIME = 180.0
PRECISION_TOPUP_GAP = 0.007
SLOW_GAP_1 = 0.05
SLOW_GAP_2 = 0.20

VALIDATE_SOLUTION = True
VALIDATION_TOL_M = 1e-3

# ─────────────────────────────────────────────────────────────────────
# RUN PROFILE
# ─────────────────────────────────────────────────────────────────────
COST_TYPES_RUN: List[str] = ["P50 - Real"]
BENEFIT_SCENARIOS: Dict[str, str] = {"LIN40": "Benefits Linear 40yrs"}

SURPLUS_OPTIONS_M: Dict[str, float] = {"s1500": 1500.0}#, "s2000": 2000.0}
PLUSMINUS_LEVELS_M: List[float] = [0,250]

# --- PROJECT CONSTRAINTS -----------------------------------------------------
FORCED_START: Dict[str, Dict] = {
    # "Project A": {"start": 2028, "include": True},
}

MIN_START_YEAR: Dict[str, int] = {
    # "Project 1": 2035,
}

# ---------------------------
PROJECT_SELECTION_MODE = "auto"
WHITELIST_FALLBACK_TO_BLACKLIST_IF_EMPTY = True
WARN_ON_UNMATCHED_RULE_NAMES = True
DIMENSION_INCLUSIONS: Dict[str, bool] = {
    "Total": True,
    "Healthy and safe people": True,
    "Inclusive Access": True,
    "Environmental Sustainability": True,
    "Economic Prosperity": True,
    "Resilience and Security": True,
}

# ─────────────────────────────────────────────────────────────────────
# MONOTONE PV GUARD BETWEEN BUFFERS
# ─────────────────────────────────────────────────────────────────────
ENFORCE_MONOTONE_PV_ACROSS_BUFFERS = True
MONO_REL_EPS = 1e-4
MONO_ABS_EPS = 1e-3

# ─────────────────────────────────────────────────────────────────────
# LOGGING / HELPERS
# ─────────────────────────────────────────────────────────────────────
def cal_years(ny: int) -> List[int]:
    return [START_FY + i for i in range(ny)]

def clean(s: str) -> str:
    return re.sub(r"\s+", " ", str(s or "").replace("\xa0", " ")).strip()

def norm(s: str) -> str:
    return clean(s).lower()

def iround(x: float, scale: float) -> int:
    return int(round(float(x) * float(scale)))

def _log(msg: str) -> None:
    if VERBOSE >= 1:
        print(msg)

class ParamChangeLogger:
    def __init__(self):
        self._last_params: Dict[str, Any] = {}

    def log_changes(self, params: Dict[str, Any], header: str = "Param changes") -> None:
        changed = {
            k: v
            for k, v in params.items()
            if k not in self._last_params or self._last_params[k] != v
        }
        if changed:
            self._last_params.update(params)
            changed_str = ", ".join(
                f"{k}={repr(v)}" for k, v in sorted(changed.items())
            )
            print(f"{header}: {changed_str}")

_PARAM_LOGGER = ParamChangeLogger()

def _apply_params_logged(m: co.Model, updates: Dict[str, Any], header: str) -> None:
    _PARAM_LOGGER.log_changes(updates, header=header)
    for k, v in updates.items():
        try:
            m.setParam(k, v)
        except Exception:
            # Silently ignore unknown/obsolete parameters on this COPT version.
            pass

def _has_incumbent(m: co.Model) -> bool:
    try:
        hm = m.getAttr(COPT.Attr.HasMipSol)
        if hm is not None:
            return bool(hm)
    except Exception:
        pass
    try:
        m.getAttr(COPT.Attr.ObjVal)
        return True
    except Exception:
        return False

def _best_gap(m: co.Model) -> Optional[float]:
    """Best known relative MIP gap (fraction, not percent, if available)."""
    try:
        g = m.getAttr(COPT.Attr.BestGap)
        if g is not None:
            return float(g)
    except Exception:
        pass
    try:
        obj = float(m.getAttr(COPT.Attr.ObjVal))
        bnd = float(m.getAttr(COPT.Attr.BestBnd))
        denom = max(1.0, abs(obj))
        return abs(bnd - obj) / denom
    except Exception:
        return None

RUN_ID = hashlib.sha1(
    f"{OPTIMISATION_PROFILE}|{SOLVER_SEED_DEFAULT}|{SOLVER_THREADS_DEFAULT}".encode(
        "utf-8"
    )
).hexdigest()[:8]

def _val(val_by_id: Optional[Dict[int, float]], var: "co.Var") -> float:
    try:
        if val_by_id is not None:
            v = val_by_id.get(id(var), None)
            if v is not None:
                return float(v)
        return float(var.X)
    except Exception:
        return 0.0

def _now_stamp() -> str:
    return time.strftime("%Y%m%d_%H%M%S", time.localtime())

def pkl_save(path: Path, payload: Dict[str, Any]) -> None:
    """
    Save payload to pickle and log a one-line summary including final gap (%)
    where available.
    """
    with path.open("wb") as f:
        pickle.dump(payload, f, protocol=pickle.HIGHEST_PROTOCOL)

    size = path.stat().st_size if path.exists() else 0
    objective = payload.get("objective", payload.get("primary_dim", "?"))
    best = payload.get("best", {}) or {}
    pv = best.get("pv", payload.get("pv_total", "?"))

    gap_frac = best.get("gap", payload.get("gap", None))
    gap_pct_val = best.get("gap_pct", payload.get("gap_pct", None))

    gap_str: str
    if isinstance(gap_pct_val, (int, float)):
        gap_str = f"{gap_pct_val:.4f}%"
    elif isinstance(gap_frac, (int, float)):
        gap_str = f"{gap_frac * 100.0:.4f}%"
    else:
        gap_str = "?"

    print(
        f"Saved: {path} ({size} bytes) "
        f"objective={objective} pv={pv} gap={gap_str}"
    )

# ─────────────────────────────────────────────────────────────────────
# I/O – Costs & Benefits
# ─────────────────────────────────────────────────────────────────────
def load_costs(cost_type: str):
    df = pd.read_excel(DATA_FILE, sheet_name="Costs", engine="openpyxl")
    df.columns = [str(c).strip() for c in df.columns]
    proj_col = [c for c in df.columns if c.lower() == "project"]
    if not proj_col:
        raise RuntimeError("Costs sheet needs 'Project'.")
    proj_col = proj_col[-1]

    horizon_all = [START_FY + i for i in range(YEARS)]
    year_cols = {int(c): c for c in df.columns if str(c).isdigit()}
    use_cols = [year_cols.get(y, None) for y in horizon_all]

    cut = df[df["Cost type"].astype(str).str.strip() == str(cost_type).strip()].copy()
    costs_input: Dict[str, List[float]] = {}
    for _, r in cut.iterrows():
        p = clean(r[proj_col])
        vals = [
            (pd.to_numeric(r[c], errors="coerce") if c is not None else 0.0)
            for c in use_cols
        ]
        costs_input[p] = (pd.Series(vals).fillna(0.0) / 1_000_000.0).tolist()  # M

    projects: Dict[str, Dict[str, Any]] = {}
    variants: Dict[str, Dict[str, Any]] = {}
    for p, seriesM in costs_input.items():
        s = pd.Series(seriesM)
        nz = s.to_numpy().nonzero()[0]
        if nz.size == 0:
            continue
        seg = s.iloc[nz.min() : nz.max() + 1].tolist()
        projects[p] = {"cost": float(sum(seg)), "dur": len(seg), "spend": seg}
        variants[p] = {
            "base": p,
            "dur": len(seg),
            "spend": seg,
            "first_year_idx": int(nz.min()),
        }

    costs_input_df = pd.DataFrame(costs_input, index=horizon_all).T
    costs_input_df.index.name = "Project"
    return projects, variants, costs_input_df

def load_benefits(sheet: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    df = pd.read_excel(DATA_FILE, sheet_name=sheet, engine="openpyxl")
    df.columns = [str(c).strip() for c in df.columns]
    if "Project" not in df.columns:
        raise RuntimeError("Benefits sheet needs 'Project'.")
    dim_col = None
    for c in df.columns:
        if c.lower().startswith("dimension"):
            dim_col = c
            break
    if dim_col is None:
        raise RuntimeError("Benefits sheet needs 'Dimension'.")
    if dim_col != "Dimension":
        df.rename(columns={dim_col: "Dimension"}, inplace=True)

    tcols: List[Tuple[int, str]] = []
    for c in df.columns:
        m = re.fullmatch(r"[tT]\s*\+\s*(\d+)", str(c))
        if m:
            tcols.append((int(m.group(1)), c))
    tcols.sort(key=lambda x: x[0])

    for _, c in tcols:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0)

    ben_kernel_df = df.copy()
    ben_kernel_df["Project"] = ben_kernel_df["Project"].map(clean)
    ben_kernel_df["Dimension"] = ben_kernel_df["Dimension"].map(clean)
    ben_kernel_df.set_index(["Project", "Dimension"], inplace=True)
    ben_kernel_df = ben_kernel_df[[c for _, c in tcols]]
    return df, ben_kernel_df

def map_benefit_kernels(benef_df: pd.DataFrame, variants: Dict[str, dict]):
    tcols = [
        c
        for _, c in sorted(
            [
                (int(re.fullmatch(r"[tT]\s*\+\s*(\d+)", c).group(1)), c)
                for c in benef_df.columns
                if re.fullmatch(r"[tT]\s*\+\s*(\d+)", c)
            ]
        )
    ]
    df = benef_df.copy()
    df["Project_clean"] = df["Project"].map(clean)
    df["Dimension_clean"] = df["Dimension"].map(clean)

    flows_by_dim: Dict[str, Dict[str, List[float]]] = {}
    order: List[str] = []
    for _, r in df.iterrows():
        p = r["Project_clean"]
        d = r["Dimension_clean"]
        seq = r[tcols].to_numpy(dtype=float).tolist()
        flows_by_dim.setdefault(d, {})[p] = seq
        if d not in order:
            order.append(d)

    if "Total" not in flows_by_dim:
        flows_by_dim["Total"] = {}
    all_dims = [d for d in order if d.lower() != "total"]
    projs = set()
    for d in all_dims:
        projs |= set(flows_by_dim[d].keys())

    for p in projs:
        acc = None
        for d in all_dims:
            v = flows_by_dim[d].get(p)
            if v is None:
                continue
            acc = v if acc is None else [a + b for a, b in zip(acc, v)]
        flows_by_dim["Total"][p] = acc or [0.0] * len(tcols)

    if "Total" not in order:
        order.append("Total")

    keeps = set(variants.keys())
    for d in list(flows_by_dim.keys()):
        flows_by_dim[d] = {p: seq for p, seq in flows_by_dim[d].items() if p in keeps}

    kernels_by_dim: Dict[str, Dict[str, List[float]]] = {}
    for d, mp_ in flows_by_dim.items():
        kernels_by_dim[d] = {}
        for v, meta in variants.items():
            dur = meta["dur"]
            ker = mp_.get(v, [])
            kernels_by_dim[d][v] = [0.0] * dur + [float(x) for x in ker]
    return order, kernels_by_dim

# ─────────────────────────────────────────────────────────────────────
# Rules / Starts
# ─────────────────────────────────────────────────────────────────────
def apply_forced_rules(variants: Dict[str, dict], rules: Dict[str, Dict]):
    v_norm2canon = {norm(v): v for v in variants.keys()}
    v_norm_set = set(v_norm2canon.keys())
    include_true_norm, exclude_true_norm = set(), set()
    start_map_all_norm: Dict[str, int] = {}

    for raw_name, spec in (rules or {}).items():
        pname_norm = norm(raw_name)
        inc = spec.get("include", None)
        st = spec.get("start", None)
        if inc is True:
            include_true_norm.add(pname_norm)
        if inc is False:
            exclude_true_norm.add(pname_norm)
        if st is not None:
            start_map_all_norm[pname_norm] = int(st)

    matched_includes_norm = include_true_norm & v_norm_set
    mode_req = (PROJECT_SELECTION_MODE or "auto").strip().lower()

    has_include_true_rules = len(include_true_norm) > 0
    use_whitelist = (mode_req == "whitelist") or (
        mode_req == "auto" and has_include_true_rules
    )

    if use_whitelist:
        keep_norm = matched_includes_norm
        mode = "WHITELIST"
    else:
        if len(matched_includes_norm) == 0 and WHITELIST_FALLBACK_TO_BLACKLIST_IF_EMPTY:
            keep_norm = v_norm_set - (exclude_true_norm & v_norm_set)
            mode = "BLACKLIST (fallback)"
        else:
            keep_norm = v_norm_set - (exclude_true_norm & v_norm_set)
            mode = "BLACKLIST" if mode_req != "auto" else "BLACKLIST (auto)"

    keep_canon = {v_norm2canon[n] for n in keep_norm}
    kept_variants = {v: variants[v] for v in variants if v in keep_canon}
    forced_exact: Dict[str, int] = {}
    for n, yr in start_map_all_norm.items():
        if n in keep_norm:
            forced_exact[v_norm2canon[n]] = int(yr)

    if VERBOSE >= 1:
        print(
            f" [rules] Mode={mode}; kept={len(kept_variants)}/{len(variants)}; "
            f"forced={len(forced_exact)}"
        )

    return kept_variants, forced_exact, use_whitelist

def allowed_starts_fine(
    variants: Dict[str, dict],
    forced_exact: Dict[str, int],
    min_starts: Dict[str, int],
    Tfine: int,
) -> Dict[str, List[int]]:
    ny = Tfine
    allowed: Dict[str, List[int]] = {}
    for v, meta in variants.items():
        dur = meta["dur"]
        if v in forced_exact and forced_exact[v] is not None:
            s = forced_exact[v] - START_FY
            allowed[v] = [s] if (0 <= s <= ny - dur) else []
            continue

        s_ear = 0
        s_lat = ny - dur

        min_yr = min_starts.get(v)
        if min_yr is not None:
            s_min_constraint = min_yr - START_FY
            s_ear = max(s_ear, s_min_constraint)

        allowed[v] = list(range(s_ear, s_lat + 1)) if s_lat >= s_ear else []
    return allowed

# ─────────────────────────────────────────────────────────────────────
# PV Coeff maps
# ─────────────────────────────────────────────────────────────────────
def coeff_map_for_dim_fine(
    variants: Dict[str, dict],
    kernels_for_dim: Dict[str, List[float]],
    allowed: Dict[str, List[int]],
    Tfine: int,
    disc_vec: np.ndarray,
) -> Dict[Tuple[str, int], float]:
    out: Dict[Tuple[str, int], float] = {}
    for v in variants.keys():
        dur = variants[v]["dur"]
        ker = kernels_for_dim.get(v, [])
        if not ker:
            continue
        for s in allowed.get(v, []):
            val = 0.0
            for k, f in enumerate(ker):
                if f == 0.0:
                    continue
                t = s + k
                if 0 <= t < Tfine:
                    val += float(f) / float(disc_vec[t])
            if val != 0.0:
                out[(v, s)] = val
    return out

def coeff_int(
    coeff_map: Dict[Tuple[str, int], float],
    scale: float = PV_SCALE,
) -> Dict[Tuple[str, int], int]:
    return {k: int(round(v * scale)) for k, v in coeff_map.items() if v != 0.0}

# ─────────────────────────────────────────────────────────────────────
# Greedy warm start
# ─────────────────────────────────────────────────────────────────────
def greedy_warm_start(
    variants: Dict[str, dict],
    allowed: Dict[str, List[int]],
    Tfine: int,
    funding_target_M: np.ndarray,  # capacity per year (not compulsory spend)
    max_starts_per_year: int,
    forced_exact: Dict[str, int],
    coeff_total_map: Dict[Tuple[str, int], float],
    reuse_sel: Optional[Dict[Tuple[str, int], int]] = None,
) -> Dict[Tuple[str, int], int]:
    ny = Tfine
    capacity_prefix = np.cumsum(funding_target_M)
    spend_cum = np.zeros(ny, dtype=float)
    starts_count = np.zeros(ny, dtype=int)
    sel: Dict[Tuple[str, int], int] = {}

    # Try to reuse previous selection if it fits the new rules/capacity
    if reuse_sel:
        ok = True
        tmp_sel: Dict[Tuple[str, int], int] = {}
        tmp_count = starts_count.copy()
        tmp_cum = spend_cum.copy()
        for (v, s), on in sorted(
            reuse_sel.items(), key=lambda kv: (kv[0][1], kv[0][0])
        ):
            if not on:
                continue
            if s not in set(allowed.get(v, [])):
                ok = False
                break
            if v not in variants:
                ok = False
                break
            if tmp_count[s] >= max_starts_per_year:
                ok = False
                break
            d = variants[v]["dur"]
            vec = np.array(variants[v]["spend"], dtype=float)
            inc = np.zeros(ny, dtype=float)
            inc[s : s + d] = vec
            if np.any(tmp_cum + np.cumsum(inc) - capacity_prefix > 1e-9):
                ok = False
                break
            tmp_cum += np.cumsum(inc)
            tmp_count[s] += 1
            tmp_sel[(v, s)] = 1
        if ok:
            sel = tmp_sel
            starts_count = tmp_count
            spend_cum = tmp_cum

    # Enforce forced projects in chronological order
    forced_order: List[Tuple[int, str]] = []
    for v, yr in (forced_exact or {}).items():
        if yr is None or v not in variants:
            continue
        s = int(yr - START_FY)
        d = variants[v]["dur"]
        if s < 0 or s > ny - d:
            continue
        if s not in set(allowed.get(v, [])):
            continue
        if (v, s) not in sel:
            forced_order.append((s, v))
    forced_order.sort()

    for s, v in forced_order:
        if starts_count[s] >= max_starts_per_year:
            continue
        d = variants[v]["dur"]
        vec = np.array(variants[v]["spend"], dtype=float)
        inc = np.zeros(ny, dtype=float)
        inc[s : s + d] = vec
        if np.any(spend_cum + np.cumsum(inc) - capacity_prefix > 1e-9):
            continue
        spend_cum += np.cumsum(inc)
        starts_count[s] += 1
        sel[(v, s)] = 1

    remain = [v for v in variants.keys() if v not in [vv for (vv, _) in sel]]
    order: List[Tuple[float, str]] = []
    for v in remain:
        denom = float(sum(variants[v]["spend"])) or 1e-9
        bestpv = max(
            (coeff_total_map.get((v, s), 0.0) for s in allowed.get(v, [])),
            default=0.0,
        )
        order.append((bestpv / denom, v))
    order.sort(reverse=True)

    for _, v in order:
        d = variants[v]["dur"]
        vec = np.array(variants[v]["spend"], dtype=float)
        pv_starts: List[Tuple[float, int]] = []
        for s in allowed.get(v, []):
            pv_starts.append((coeff_total_map.get((v, s), 0.0), s))
        pv_starts.sort(key=lambda x: x[0], reverse=True)

        for pv, s in pv_starts:
            if starts_count[s] >= max_starts_per_year:
                continue
            inc = np.zeros(ny, dtype=float)
            inc[s : s + d] = vec
            if np.any(spend_cum + np.cumsum(inc) - capacity_prefix > 1e-9):
                continue
            spend_cum += np.cumsum(inc)
            starts_count[s] += 1
            sel[(v, s)] = 1
            break

    return sel

# ─────────────────────────────────────────────────────────────────────
# MODEL
# ─────────────────────────────────────────────────────────────────────
class CoptSpendMatchMO:
    """
    Spend / funding model with:
      - funding[t] FIXED to envelope[t] if active.
      - PIECEWISE SOFT CAP on net[t] (tiers of penalties).
      - objective = minimize backlog + penalties - PV reward
      - MUST PURCHASE EVERY PROJECT (Hard start constraint = 1.0)
      - DIVIDEND[t]: Restricted to end-of-life (y[t+1]=0).
    """

    def __init__(
        self,
        variants: Dict[str, dict],
        allowed: Dict[str, List[int]],
        funding_target_S: np.ndarray,  # scaled per-year capacity
        Tn: int,
        taper_start_idx: int,  # legacy; not used by new logic
        spend_by_year: Dict[str, List[float]],
        max_starts_per_year: int,
        is_whitelist_model: bool,
        env: Optional[co.Envr] = None,
        name: str = "spend_match_mo",
        *,
        relax_binaries: bool = False,
    ):
        self.Tn = int(Tn)
        self.taper_start_idx = taper_start_idx

        self.env = env or co.Envr()
        self.m: co.Model = self.env.createModel(name)
        init_params = {
            "RandSeed": int(SOLVER_SEED_DEFAULT),
            "Threads": int(SOLVER_THREADS_DEFAULT),
            "MipStartMode": 2,
            "Logging": 1,
            "Display": 1,
            "Presolve": 1,
            "Scaling": 1,
        }
        _apply_params_logged(self.m, init_params, header="Model init params")

        # Funding capacity (scaled) per year – envelope path
        self.funding_target_S = funding_target_S

        # Decision x[v,s]
        self.x: Dict[Tuple[str, int], co.Var] = {}
        self.by_year: Dict[int, List[Tuple[str, int]]] = defaultdict(list)
        x_vtype = COPT.CONTINUOUS if relax_binaries else COPT.BINARY
        for v in variants.keys():
            for s in allowed.get(v, []):
                var = self.m.addVar(lb=0.0, ub=1.0, vtype=x_vtype, name=f"x[{v}|{s}]")
                self.x[(v, s)] = var
                if 0 <= s < self.Tn:
                    self.by_year[s].append((v, s))

        # Envelope-active indicator y[t]; monotone non-increasing.
        y_vtype = COPT.CONTINUOUS if relax_binaries else COPT.BINARY
        self.y: List[co.Var] = [
            self.m.addVar(lb=0.0, ub=1.0, vtype=y_vtype, name=f"active[{t}]")
            for t in range(self.Tn)
        ]
        for t in range(self.Tn - 1):
            self.m.addConstr(self.y[t] >= self.y[t + 1], name=f"active_noninc[{t}]")

        # Spend convolution (scaled)
        spend_terms: List[List[Tuple[co.Var, int]]] = [[] for _ in range(self.Tn)]
        for (v, s), var in self.x.items():
            vec = spend_by_year[v]
            for k, amtM in enumerate(vec):
                if amtM == 0.0:
                    continue
                t = s + k
                if 0 <= t < self.Tn:
                    spend_terms[t].append((var, iround(amtM, SPEND_SCALE)))

        self.spend_expr: List[co.LinExpr] = []
        for t in range(self.Tn):
            if spend_terms[t]:
                self.spend_expr.append(
                    co.quicksum(coeff * var for (var, coeff) in spend_terms[t])
                )
            else:
                self.spend_expr.append(co.LinExpr(0.0))

        # If there is spend in year t from some x[v,s], then y[t] must be 1.
        for t in range(self.Tn):
            for (var, _coeff) in spend_terms[t]:
                self.m.addConstr(self.y[t] >= var)

        # Funding actually drawn each year (scaled units).
        # FIXED: Must equal the envelope if y[t] is 1.
        self.funding: List[co.Var] = []
        for t in range(self.Tn):
            ub = float(self.funding_target_S[t])
            var = self.m.addVar(lb=0.0, ub=ub, vtype=COPT.CONTINUOUS, name=f"fund[{t}]")
            self.funding.append(var)
            # funding[t] == y[t] * envelope_t
            self.m.addConstr(var == self.y[t] * ub, name=f"fund_fixed[{t}]")

        # -- DIVIDEND VARIABLE (Restricted to End-of-Life) --
        self.dividend: List[co.Var] = [
            self.m.addVar(lb=0.0, vtype=COPT.CONTINUOUS, name=f"dividend[{t}]")
            for t in range(self.Tn)
        ]
        net_bigM = float(np.sum(self.funding_target_S))

        # NEW CONSTRAINT: RESTRICT DIVIDEND
        # If y[t+1] == 1, then dividend[t] must be 0.
        for t in range(self.Tn - 1):
            self.m.addConstr(
                self.dividend[t] <= net_bigM * (1.0 - self.y[t + 1]),
                name=f"div_restrict[{t}]",
            )
        # Last year (Tn-1) is always allowed to dividend (y[Tn] is implicitly 0).

        # Net path (non-negative; no debt).
        self.net: List[co.Var] = [
            self.m.addVar(lb=0.0, vtype=COPT.CONTINUOUS, name=f"net[{t}]")
            for t in range(self.Tn)
        ]

        # Year 0: Net[0] = Fund[0] - Spend[0] - Dividend[0]
        self.m.addConstr(
            self.net[0] == self.funding[0] - self.spend_expr[0] - self.dividend[0],
            name="net0",
        )
        # Subsequent: Net[t] = Net[t-1] + Fund[t] - Spend[t] - Dividend[t]
        for t in range(1, self.Tn):
            self.m.addConstr(
                self.net[t]
                == self.net[t - 1]
                + self.funding[t]
                - self.spend_expr[t]
                - self.dividend[t],
                name=f"net[{t}]",
            )

        # ------------------------------------------------------------------
        # PIECEWISE LINEAR SOFT CAP
        # ------------------------------------------------------------------
        self.excess_tiers: List[List[co.Var]] = [[] for _ in range(self.Tn)]
        self.tier_penalties_expr = co.LinExpr(0.0)

        base_threshold = PIECEWISE_CAP_TIERS[0][0]

        for t in range(self.Tn):
            env_S = float(self.funding_target_S[t])
            base_cap_S = env_S * base_threshold

            tier_vars_t: List[co.Var] = []
            for i, (thresh_start, weight) in enumerate(PIECEWISE_CAP_TIERS):
                is_last = i == len(PIECEWISE_CAP_TIERS) - 1
                if not is_last:
                    thresh_next = PIECEWISE_CAP_TIERS[i + 1][0]
                    width_S = env_S * (thresh_next - thresh_start)
                    v_tier = self.m.addVar(
                        lb=0.0,
                        ub=width_S,
                        vtype=COPT.CONTINUOUS,
                        name=f"exc_t{i}_{t}",
                    )
                else:
                    v_tier = self.m.addVar(
                        lb=0.0, vtype=COPT.CONTINUOUS, name=f"exc_t{i}_{t}"
                    )

                tier_vars_t.append(v_tier)
                self.tier_penalties_expr.addTerm(v_tier, weight)

            self.excess_tiers[t] = tier_vars_t

            if t < self.Tn - 1:
                sum_excess_t = co.quicksum(tier_vars_t)
                self.m.addConstr(
                    self.net[t]
                    <= base_cap_S
                    + sum_excess_t
                    + net_bigM * (1.0 - self.y[t + 1]),
                    name=f"net_cap_piecewise[{t}]",
                )
            else:
                # Last year cleanup – no excess tiers; just cap by net_bigM.
                for v in tier_vars_t:
                    self.m.addConstr(v == 0.0)
                self.m.addConstr(self.net[t] <= net_bigM, name=f"net_cap_last[{t}]")

        # Auxiliary backlog variables
        self.backlog: List[co.Var] = [
            self.m.addVar(lb=0.0, vtype=COPT.CONTINUOUS, name=f"backlog[{t}]")
            for t in range(self.Tn)
        ]
        for t in range(self.Tn):
            if t < self.Tn - 1:
                self.m.addConstr(
                    self.backlog[t]
                    >= self.net[t] - net_bigM * (1.0 - self.y[t + 1]),
                    name=f"backlog_lb_link[{t}]",
                )
                self.m.addConstr(
                    self.backlog[t]
                    <= self.net[t] + net_bigM * (1.0 - self.y[t + 1]),
                    name=f"backlog_ub_link[{t}]",
                )
            else:
                self.m.addConstr(
                    self.backlog[t] == 0.0,
                    name="backlog_last_zero",
                )

        # Annual backlog sum
        self.annual_backlog_sum = co.quicksum(self.backlog[t] for t in range(self.Tn))

        # Starts: per-project caps; per-year caps
        start_constr = (COPT.EQUAL, 1.0)

        if not relax_binaries:
            for v in {vv for vv, _ in self.x}:
                cols = [self.x[(vv, s)] for (vv, s) in self.x if vv == v]
                if cols:
                    self.m.addConstr(
                        co.quicksum(cols),
                        start_constr[0],
                        start_constr[1],
                        name=f"start_once[{v}]",
                    )
            for t in range(self.Tn):
                cols = [self.x[(v, s)] for (v, s) in self.by_year.get(t, [])]
                if cols:
                    self.m.addConstr(
                        co.quicksum(cols) <= int(max_starts_per_year),
                        name=f"cap_starts[{t}]",
                    )
        else:
            for v in {vv for vv, _ in self.x}:
                cols = [self.x[(vv, s)] for (vv, s) in self.x if vv == v]
                if cols:
                    self.m.addConstr(
                        co.quicksum(cols),
                        start_constr[0],
                        start_constr[1],
                        name=f"start_once_relax[{v}]",
                    )
            for t in range(self.Tn):
                cols = [self.x[(v, s)] for (v, s) in self.by_year.get(t, [])]
                if cols:
                    self.m.addConstr(
                        co.quicksum(cols) <= float(max_starts_per_year),
                        name=f"cap_starts_relax[{t}]",
                    )

        if self.x:
            self.m.addConstr(
                co.quicksum(self.x.values()) >= 1.0,
                name="at_least_one_project",
            )

        self._obj_cache: Dict[str, co.LinExpr] = {}

    # ---- Wrappers -----------------------------------------------------------
    def obj_expr(
        self,
        coeff_int_map: Dict[Tuple[str, int], int],
        cache_key: Optional[str] = None,
    ) -> co.LinExpr:
        if cache_key and cache_key in self._obj_cache:
            return self._obj_cache[cache_key]
        terms = []
        for (v, s), w in coeff_int_map.items():
            var = self.x.get((v, s))
            if var is not None and int(w) != 0:
                terms.append(w * var)
        expr = co.quicksum(terms) if terms else co.LinExpr(0.0)
        if cache_key:
            self._obj_cache[cache_key] = expr
        return expr

    def add_floor(self, expr: co.LinExpr, target_unscaled: float, name: str) -> None:
        self.m.addConstr(
            expr >= int(math.floor(target_unscaled * PV_SCALE)),
            name=name,
        )

    def add_hint_from_starts(self, sel: Dict[Tuple[str, int], int]) -> None:
        if not sel:
            return
        try:
            vars_: List[co.Var] = []
            vals_: List[float] = []
            for (k, on) in sel.items():
                if on and k in self.x:
                    vars_.append(self.x[k])
                    vals_.append(1.0)
            if vars_:
                self.m.addMIPStart(vars_, vals_)
        except Exception:
            pass

# ─────────────────────────────────────────────────────────────────────
# Solve helpers
# ─────────────────────────────────────────────────────────────────────
@dataclass
class SolveResult:
    status: int
    has_inc: bool
    gap: Optional[float]
    seconds: float
    victory_lap_triggered: bool = False

class Phase2Callback(co.CallbackBase):
    """
    Dynamic Victory Lap callback:
      - Monitors BestObj / BestBnd via getInfo(COPT.CbInfo.*).
      - If gap < target_gap: start a countdown (victory_lap_duration).
      - Otherwise: interrupts after normal_time_limit.
    """

    def __init__(
        self,
        normal_time_limit: float,
        target_gap: float,
        victory_lap_duration: float = 60.0,
    ):
        super().__init__()
        self.normal_time_limit = float(normal_time_limit)
        self.target_gap = float(target_gap)
        self.victory_lap_duration = float(victory_lap_duration)
        self.start_time = time.time()
        self.victory_start_time: Optional[float] = None
        self.triggered_victory: bool = False

    def callback(self) -> None:
        # Only respond in MIP solution / node contexts (if we could detect them)
        where = self.getWhere()
        valid_contexts: List[int] = []
        if CTX_MIPSOL:
            valid_contexts.append(CTX_MIPSOL)
        if CTX_MIPNODE:
            valid_contexts.append(CTX_MIPNODE)
        if valid_contexts and where not in valid_contexts:
            return

        now = time.time()
        elapsed = now - self.start_time

        # If we already triggered the Victory Lap, just watch the timer.
        if self.triggered_victory:
            if (
                self.victory_start_time is not None
                and now - self.victory_start_time >= self.victory_lap_duration
            ):
                self.interrupt()
            return

        # Pull BestObj / BestBnd from COPT
        try:
            best_obj = float(self.getInfo(COPT.CbInfo.BestObj))
            best_bnd = float(self.getInfo(COPT.CbInfo.BestBnd))
        except Exception:
            return

        # Check if we actually have an incumbent yet.
        if not math.isfinite(best_obj):
            return
        try:
            if abs(best_obj) >= 0.5 * float(COPT.INFINITY):
                return
        except Exception:
            # If INFINITY is not available or weird, fall back to just finite check.
            pass

        denom = max(1.0, abs(best_obj))
        current_gap = abs(best_bnd - best_obj) / denom

        # Victory trigger
        if current_gap <= self.target_gap:
            self.triggered_victory = True
            self.victory_start_time = now
            return

        # Normal time cutoff (only if we haven't triggered victory)
        if elapsed >= self.normal_time_limit:
            self.interrupt()

# --- Profile-specific solve strategies --------------------------------------
def _solve_single_phase(
    m: co.Model, stage: str, base_time: float, rel_gap: float
) -> SolveResult:
    """
    v94.11-style single-phase solve with optional precision top-up.
    Used for 'fast' and 'balanced' profiles.
    """
    t0 = time.time()
    _log(f"[{stage}] Single-phase solve: {base_time:.0f}s, target gap={rel_gap:.4f}")
    _apply_params_logged(
        m,
        {"RelGap": float(rel_gap), "TimeLimit": float(base_time)},
        header=f"{stage} main settings",
    )
    m.solve()

    has_inc = _has_incumbent(m)
    gap = _best_gap(m)

    # Optional precision top-up if we have an incumbent but the gap is still loose.
    if has_inc and (gap is None or gap > PRECISION_TOPUP_GAP):
        elapsed = time.time() - t0
        remaining_time = max(0.0, base_time - elapsed)
        top_up_time = max(remaining_time, PRECISION_TOPUP_TIME)
        if top_up_time > 5.0:
            _log(
                f"[{stage}] Precision top-up: +{top_up_time:.0f}s, "
                f"target gap={PRECISION_TOPUP_GAP:.4f}"
            )
            _apply_params_logged(
                m,
                {
                    "TimeLimit": float(top_up_time),
                    "RelGap": float(PRECISION_TOPUP_GAP),
                },
                header=f"{stage} top-up settings",
            )
            m.solve()
            has_inc = _has_incumbent(m)
            gap = _best_gap(m)

    final_status = int(m.status)
    seconds = time.time() - t0
    if final_status == COPT.INFEASIBLE:
        _log(f"[{stage}] Solve status: INFEASIBLE.")
        return SolveResult(
            status=final_status, has_inc=False, gap=None, seconds=seconds
        )

    return SolveResult(
        status=final_status, has_inc=has_inc, gap=gap, seconds=seconds
    )

def _solve_two_phase(
    m: co.Model, stage: str, base_time: float, rel_gap: float
) -> SolveResult:
    """
    2-Phase: Probe + Verify+VictoryLap (for 'thorough' profile).
    """
    t_start = time.time()
    t_probe = max(5.0, base_time * 0.10)
    t_verify = max(10.0, base_time - t_probe)

    # Phase 1: Probe
    _log(f"[{stage}] === Phase 1/2: Probe ({t_probe:.0f}s) ===")
    _apply_params_logged(
        m,
        {
            "TimeLimit": float(t_probe),
            "RelGap": float(rel_gap),
        },
        header=f"{stage} Probe Params",
    )
    m.solve()

    # Early exit if we already reached the target gap.
    if _has_incumbent(m):
        g = _best_gap(m)
        if g is not None and g <= rel_gap:
            seconds = time.time() - t_start
            _log(
                f"[{stage}] Probe hit target gap {g:.4%} ≤ {rel_gap:.4%}, "
                "skipping Verify."
            )
            return SolveResult(
                status=int(m.status),
                has_inc=True,
                gap=g,
                seconds=seconds,
            )

    # Phase 2: Verify + Victory Lap
    _log(
        f"[{stage}] === Phase 2/2: Verify ({t_verify:.0f}s) + Dynamic Victory Lap ==="
    )
    _apply_params_logged(
        m,
        {
            "TimeLimit": float(COPT.INFINITY),
            "RelGap": 0.0,  # let callback drive termination
        },
        header=f"{stage} Verify Params",
    )

    cb = Phase2Callback(normal_time_limit=t_verify, target_gap=rel_gap)

    # Attach callback; if unsupported, fall back to single-phase behaviour.
    try:
        context_mask = 0
        if CTX_MIPSOL:
            context_mask |= CTX_MIPSOL
        if CTX_MIPNODE:
            context_mask |= CTX_MIPNODE
        if context_mask:
            m.setCallback(cb, context_mask)
        else:
            m.setCallback(cb)
    except TypeError:
        try:
            m.setCallback(cb)
        except Exception:
            _log(
                f"[{stage}] Callback not supported; falling back to "
                "single-phase solver."
            )
            return _solve_single_phase(m, stage, base_time, rel_gap)

    m.solve()
    # Try to clear the callback (API differences tolerated).
    try:
        m.setCallback(None, 0)
    except Exception:
        try:
            m.setCallback(None)
        except Exception:
            pass

    final_status = int(m.status)
    seconds = time.time() - t_start
    if final_status == COPT.INFEASIBLE:
        _log(f"[{stage}] Solve status: INFEASIBLE.")
        return SolveResult(
            status=final_status,
            has_inc=False,
            gap=None,
            seconds=seconds,
            victory_lap_triggered=cb.triggered_victory,
        )

    gap = _best_gap(m)
    if cb.triggered_victory:
        _log(
            f"[{stage}] Victory Lap triggered; final gap: "
            f"{gap if gap is not None else float('nan'):.4%}"
        )

    return SolveResult(
        status=final_status,
        has_inc=_has_incumbent(m),
        gap=gap,
        seconds=seconds,
        victory_lap_triggered=cb.triggered_victory,
    )

def _solve_three_phase(
    m: co.Model, stage: str, base_time: float, rel_gap: float
) -> SolveResult:
    """
    3-Phase: Probe + Verify+VictoryLap + Grind (for 'ultra' profile).
    """
    t_start = time.time()

    # Time splits
    t_probe = max(5.0, base_time * 0.10)
    t_verify = max(10.0, base_time * 0.30)

    # -------------------------------------------------------------------------
    # Phase 1: Probe
    # -------------------------------------------------------------------------
    _log(f"[{stage}] === Phase 1/3: Probe ({t_probe:.0f}s) ===")
    _apply_params_logged(
        m,
        {
            "TimeLimit": float(t_probe),
            "RelGap": float(rel_gap),
        },
        header=f"{stage} Probe Params",
    )
    m.solve()

    if _has_incumbent(m):
        g = _best_gap(m)
        if g is not None and g <= rel_gap:
            seconds = time.time() - t_start
            _log(
                f"[{stage}] Probe hit target gap {g:.4%} ≤ {rel_gap:.4%}, "
                "skipping Verify & Grind."
            )
            return SolveResult(
                status=int(m.status),
                has_inc=True,
                gap=g,
                seconds=seconds,
            )

    # -------------------------------------------------------------------------
    # Phase 2: Verify (with Victory Lap)
    # -------------------------------------------------------------------------
    _log(
        f"[{stage}] === Phase 2/3: Verify ({t_verify:.0f}s) + Dynamic Victory Lap ==="
    )
    _apply_params_logged(
        m,
        {
            "TimeLimit": float(COPT.INFINITY),
            "RelGap": 0.0,
        },
        header=f"{stage} Verify Params",
    )

    cb = Phase2Callback(normal_time_limit=t_verify, target_gap=rel_gap)

    try:
        context_mask = 0
        if CTX_MIPSOL:
            context_mask |= CTX_MIPSOL
        if CTX_MIPNODE:
            context_mask |= CTX_MIPNODE
        if context_mask:
            m.setCallback(cb, context_mask)
        else:
            m.setCallback(cb)
    except TypeError:
        try:
            m.setCallback(cb)
        except Exception:
            _log(
                f"[{stage}] Callback not supported; falling back to single-phase "
                "solver."
            )
            return _solve_single_phase(m, stage, base_time, rel_gap)

    m.solve()
    try:
        m.setCallback(None, 0)
    except Exception:
        try:
            m.setCallback(None)
        except Exception:
            pass

    current_gap = _best_gap(m)
    if cb.triggered_victory:
        seconds = time.time() - t_start
        _log(
            f"[{stage}] Victory Lap triggered & completed. Gap: "
            f"{current_gap if current_gap is not None else float('nan'):.4%} "
            f"< {rel_gap:.4%}. Skipping Grind."
        )
        return SolveResult(
            status=int(m.status),
            has_inc=_has_incumbent(m),
            gap=current_gap,
            seconds=seconds,
            victory_lap_triggered=True,
        )

    # -------------------------------------------------------------------------
    # Phase 3: Grind (remaining budget)
    # -------------------------------------------------------------------------
    elapsed = time.time() - t_start
    remaining = max(10.0, base_time - elapsed)

    _log(f"[{stage}] === Phase 3/3: Grind ({remaining:.0f}s) ===")
    _apply_params_logged(
        m,
        {
            "TimeLimit": float(remaining),
            "RelGap": float(rel_gap),
        },
        header=f"{stage} Grind Params",
    )
    m.solve()

    final_status = int(m.status)
    seconds = time.time() - t_start
    if final_status == COPT.INFEASIBLE:
        _log(f"[{stage}] Solve status: INFEASIBLE.")
        return SolveResult(
            status=final_status, has_inc=False, gap=None, seconds=seconds
        )

    return SolveResult(
        status=final_status,
        has_inc=_has_incumbent(m),
        gap=_best_gap(m),
        seconds=seconds,
        victory_lap_triggered=False,
    )

def solve_model(
    m: co.Model,
    stage: str,
    base_time: float,
    rel_gap: float,
) -> SolveResult:
    """
    Profile-aware wrapper:

      - 'fast', 'balanced'  → single-phase (v94.11-style)
      - 'thorough'          → 2-phase (Probe + Verify + Victory Lap)
      - 'ultra'             → 3-phase (Probe + Verify + Victory Lap + Grind)
    """
    profile = (OPTIMISATION_PROFILE or "thorough").strip().lower()
    if profile in ("fast", "balanced"):
        return _solve_single_phase(m, stage, base_time, rel_gap)
    elif profile == "thorough":
        return _solve_two_phase(m, stage, base_time, rel_gap)
    elif profile == "ultra":
        return _solve_three_phase(m, stage, base_time, rel_gap)
    else:
        _log(
            f"[{stage}] Unknown OPTIMISATION_PROFILE={OPTIMISATION_PROFILE!r}; "
            "falling back to single-phase strategy."
        )
        return _solve_single_phase(m, stage, base_time, rel_gap)

# ─────────────────────────────────────────────────────────────────────
# Selection / PV helpers
# ─────────────────────────────────────────────────────────────────────
def selection_from_vars(
    xmap: Dict[Tuple[str, int], co.Var]
) -> Dict[Tuple[str, int], int]:
    out: Dict[Tuple[str, int], int] = {}
    for (k, var) in xmap.items():
        try:
            val = float(var.X)
        except Exception:
            val = 0.0
        if val > 0.5:
            out[k] = 1
    return out

def selection_from_values(
    xmap: Dict[Tuple[str, int], co.Var],
    val_by_id: Optional[Dict[int, float]],
) -> Dict[Tuple[str, int], int]:
    out: Dict[Tuple[str, int], int] = {}
    for key, var in xmap.items():
        if _val(val_by_id, var) > 0.5:
            out[key] = 1
    return out

def pv_from_selection(
    coeff_map: Dict[Tuple[str, int], float],
    sel: Dict[Tuple[str, int], int],
) -> float:
    return float(sum(coeff_map.get(k, 0.0) for k, on in sel.items() if on))

def extract_solution_values(M: CoptSpendMatchMO) -> Optional[Dict[int, float]]:
    if not _has_incumbent(M.m):
        return None

    val_by_id: Dict[int, float] = {}
    all_vars: List[co.Var] = []

    all_vars.extend(M.x.values())
    if hasattr(M, "net"):
        all_vars.extend(M.net)
    if hasattr(M, "funding"):
        all_vars.extend(M.funding)
    if hasattr(M, "y"):
        all_vars.extend(M.y)
    if hasattr(M, "backlog"):
        all_vars.extend(M.backlog)
    if hasattr(M, "dividend"):
        all_vars.extend(M.dividend)
    if hasattr(M, "excess_tiers"):
        for t in range(M.Tn):
            all_vars.extend(M.excess_tiers[t])

    for var in all_vars:
        if var is not None:
            val_by_id[id(var)] = _val(None, var)
    return val_by_id

# ─────────────────────────────────────────────────────────────────────
# Diagnostics / PKL
# ─────────────────────────────────────────────────────────────────────
def assert_and_log_invariants(
    M: CoptSpendMatchMO,
    funding_target_M: np.ndarray,
    val_by_id: Dict[int, float],
    *,
    label: str,
) -> None:
    fy = cal_years(M.Tn)
    net = np.array([_val(val_by_id, M.net[t]) for t in range(M.Tn)], dtype=float)

    # Check no-debt rule
    if np.any(net < -1e-5):  # Allow for small numerical noise
        viol_t = int(np.argmin(net))
        raise AssertionError(
            f"[{label}] debt detected at year {fy[viol_t]}! "
            f"Net balance went negative: {net[viol_t]/SPEND_SCALE:.3f} M"
        )

    print(f"[diag] {label}:")
    print(
        f"[diag] head net="
        f"{[(fy[i], round(net[i]/SPEND_SCALE, 3)) for i in range(min(5, M.Tn))]} "
        f"tail net="
        f"{[(fy[i], round(net[i]/SPEND_SCALE, 3)) for i in range(max(0, M.Tn-5), M.Tn)]}"
    )
    print(
        f"[diag] Closing Net Balance (raw model): {net[-1]/SPEND_SCALE:,.2f} M"
    )

def dump_pickle_full(
    M: CoptSpendMatchMO,
    tag: str,
    *,
    projects: Dict[str, dict],
    variants: Dict[str, dict],
    costs_input_df: pd.DataFrame,
    ben_kernel_df: pd.DataFrame,
    kernels_by_dim: Dict[str, Dict[str, List[float]]],
    benefit_rate: float,
    scenario_name: str,
    primary_dim: str,
    Tfine: int,
    funding_target_M: np.ndarray,  # envelope capacity path (M)
    sel_override: Optional[Dict[Tuple[str, int], int]] = None,
    val_override: Optional[Dict[int, float]] = None,
    gap_override: Optional[float] = None,
    extra_diag: Optional[Dict[str, str]] = None,
    status_override: Optional[str] = None,
) -> None:
    """
    Build all derived tables for a given model + selection and write them to a
    pickle file. Also records the final relative gap (fraction and percent).
    """
    ny = Tfine
    fy = cal_years(ny)

    # Selection
    if sel_override is not None:
        sel = sel_override
    elif val_override is not None:
        sel = selection_from_values(M.x, val_override)
    else:
        sel = selection_from_vars(M.x)

    status_text = status_override or "OK"
    if not sel:
        fn = CACHE / f"{(PKL_PREFIX or '')}{tag}_noSol.pkl"
        payload = {
            "status": "NoSolve",
            "reason": "no selected starts",
            "objective": primary_dim,
            "created_at": _now_stamp(),
            "gap": None,
            "gap_pct": None,
        }
        if extra_diag:
            payload["diagnostic"] = extra_diag
        pkl_save(fn, payload)
        return

    # Schedule table
    rows = []
    for (v, s) in sorted(sel.keys()):
        rows.append(
            {
                "Project": v,
                "StartFY": START_FY + s,
                "EndFY": START_FY + s + variants[v]["dur"] - 1,
                "Dur": variants[v]["dur"],
                "Scenario": scenario_name,
                "PrimaryDim": primary_dim,
            }
        )
    df_sched = pd.DataFrame(rows).sort_values(
        ["StartFY", "Project"], ignore_index=True
    )

    # Spend per project, per year (M)
    df_sp = pd.DataFrame(0.0, index=list(projects.keys()), columns=fy)
    for (v, s) in sel.keys():
        vec = variants[v]["spend"]
        for i, amt in enumerate(vec):
            t = s + i
            if 0 <= t < ny:
                df_sp.loc[v, fy[t]] += float(amt)
    df_sp.loc["Total Spend"] = df_sp.sum()

    # Determine last spend year (for reporting / closure)
    total_spend_series = df_sp.loc["Total Spend"]
    last_spend_year = None
    for yr in reversed(fy):
        if abs(float(total_spend_series.get(yr, 0.0))) > 1e-6:
            last_spend_year = yr
            break
    if last_spend_year is None:
        last_spend_idx = -1
    else:
        last_spend_idx = fy.index(last_spend_year)

    # Funding/net paths from the model
    val_by_id = val_override or {}
    funding_M_raw: List[float] = []
    if val_by_id and hasattr(M, "funding"):
        for t in range(ny):
            funding_M_raw.append(_val(val_by_id, M.funding[t]) / SPEND_SCALE)
    else:
        funding_M_raw = funding_target_M.tolist()[:ny]

    dividend_M_raw: List[float] = []
    if val_by_id and hasattr(M, "dividend"):
        for t in range(ny):
            dividend_M_raw.append(_val(val_by_id, M.dividend[t]) / SPEND_SCALE)
    else:
        dividend_M_raw = [0.0] * ny

    net_M_raw: List[float] = []
    if val_by_id and hasattr(M, "net"):
        for t in range(ny):
            net_M_raw.append(_val(val_by_id, M.net[t]) / SPEND_SCALE)
    else:
        net_prev = 0.0
        for t in range(ny):
            yr = fy[t]
            spend = float(df_sp.loc["Total Spend", yr])
            net_now = net_prev + funding_M_raw[t] - spend - dividend_M_raw[t]
            net_M_raw.append(net_now)
            net_prev = net_now

    # For reporting: after the last year with spend, envelope & funding & net are zero.
    funding_M: List[float] = []
    net_M: List[float] = []
    env_M_plot: List[float] = []
    div_M: List[float] = []
    for t in range(ny):
        if last_spend_idx >= 0 and t <= last_spend_idx:
            fund = funding_M_raw[t]
            netv = net_M_raw[t]
            env = funding_target_M[t]
            div = dividend_M_raw[t]
        else:
            fund = 0.0
            netv = 0.0
            env = 0.0
            div = 0.0
        funding_M.append(fund)
        net_M.append(netv)
        env_M_plot.append(env)
        div_M.append(div)

    # Cash-flow table (M)
    cash_rows = []
    for t in range(ny):
        yr = fy[t]
        spend = float(df_sp.loc["Total Spend", yr])
        env = env_M_plot[t]
        fund = funding_M[t]
        opening_net = 0.0 if t == 0 else net_M[t - 1]
        closing_net = net_M[t]
        div = div_M[t]
        cash_rows.append(
            dict(
                Year=yr,
                Envelope=env,
                Funding=fund,
                OpeningNet=opening_net,
                Spend=spend,
                Dividend=div,
                ClosingNet=closing_net,
                OpeningCash=max(opening_net, 0.0),
                OpeningDebt=max(-opening_net, 0.0),
                ClosingCash=max(closing_net, 0.0),
                ClosingDebt=max(-closing_net, 0.0),
            )
        )
    df_cash = pd.DataFrame(cash_rows)

    # Benefits by year / proj-dimension
    dims = list(kernels_by_dim.keys())
    Tstore = ny
    for (v, s) in sel.keys():
        for d in dims:
            ker = kernels_by_dim.get(d, {}).get(v, [])
            if ker:
                Tstore = max(Tstore, s + len(ker))
    fy_store = cal_years(Tstore)

    ben_total_by_dim_store: Dict[str, np.ndarray] = {
        d: np.zeros(Tstore, float) for d in dims
    }
    proj_dim_year_store: Dict[Tuple[str, str], np.ndarray] = {
        (p, d): np.zeros(Tstore, float) for d in dims for p in projects.keys()
    }

    for (v, s) in sel.keys():
        for d in dims:
            ker = kernels_by_dim[d].get(v, [])
            for k, f in enumerate(ker):
                t = s + k
                if 0 <= t < Tstore:
                    ben_total_by_dim_store[d][t] += float(f)
                    proj_dim_year_store[(v, d)][t] += float(f)

    df_ben_year = pd.DataFrame(
        {"Year": fy_store, **{d: ben_total_by_dim_store[d] for d in dims}}
    )

    idx = pd.MultiIndex.from_product(
        [list(projects.keys()), dims],
        names=["Project", "Dimension"],
    )
    df_ben_proj_dim_year = pd.DataFrame(0.0, index=idx, columns=fy_store)
    for (p, d), vec in proj_dim_year_store.items():
        df_ben_proj_dim_year.loc[(p, d), :] = vec

    disc = np.array([(1.0 + benefit_rate) ** t for t in range(ny)], float)
    pv_by_dim: Dict[str, float] = {
        d: float(np.sum(ben_total_by_dim_store[d][:ny] / disc)) for d in dims
    }
    pv_by_proj_dim = pd.DataFrame(0.0, index=idx, columns=["PV"])
    for (p, d) in idx:
        vec_full = df_ben_proj_dim_year.loc[(p, d)].to_numpy(dtype=float)
        pv_by_proj_dim.loc[(p, d), "PV"] = float(np.sum(vec_full[:ny] / disc))

    total_pv = pv_by_dim.get(
        "Total",
        float(np.sum([pv_by_dim[d] for d in dims if d != "Total"])),
    )

    # Diagnostics: no-debt + closing net (raw)
    try:
        val_for_diag = val_override or {id(var): float(var.X) for var in M.net}
        assert_and_log_invariants(M, funding_target_M, val_for_diag, label=tag)
        net_list = [_val(val_for_diag, M.net[t]) / SPEND_SCALE for t in range(M.Tn)]
        diag_caps: Dict[str, Any] = {
            "closing_net_M_raw": float(net_list[-1]),
            "max_net_M_raw": float(max(net_list) if net_list else 0.0),
        }
    except AssertionError as e:
        diag_caps = {"assertion_error": str(e)}

    # Final relative gap (fraction + percent)
    gap_now = float(gap_override) if gap_override is not None else _best_gap(M.m)
    gap_pct = gap_now * 100.0 if gap_now is not None else None

    out: Dict[str, Any] = dict(
        status=status_text,
        objective=primary_dim,
        created_at=_now_stamp(),
        scenario=scenario_name,
        primary_dim=primary_dim,
        schedule=df_sched,
        spend=df_sp,
        cash_flow=df_cash,
        envelope=pd.DataFrame({"Year": fy, "Envelope": env_M_plot}),
        benefits_by_year=df_ben_year,
        benefits_by_project_dimension_by_year=df_ben_proj_dim_year,
        pv_by_dimension=pv_by_dim,
        pv_total=total_pv,
        gap=gap_now,
        gap_pct=gap_pct,
        best={"pv": total_pv, "gap": gap_now, "gap_pct": gap_pct},
        pv_by_project_and_dimension=pv_by_proj_dim,
        calendar=dict(start_fy=START_FY, years=ny),
        meta=dict(
            full_envelope_M=float(funding_target_M[0])
            if len(funding_target_M) > 0
            else 0.0,
            taper_years=0,
            backlog_weight=BACKLOG_WEIGHT,
            pv_weight=PV_WEIGHT,
            net_cap_tiers=PIECEWISE_CAP_TIERS,
            run_id=RUN_ID,
        ),
    )
    if extra_diag:
        out.setdefault("diagnostic", {}).update(extra_diag)
    if diag_caps:
        out.setdefault("diagnostic", {}).update(diag_caps)

    fn = CACHE / f"{(PKL_PREFIX or '')}{tag}.pkl"
    if str(fn).endswith("}.pkl"):
        fn = Path(str(fn)[:-5] + ".pkl")
    pkl_save(fn, out)

# ─────────────────────────────────────────────────────────────────────
# Cross‑buffer registry
# ─────────────────────────────────────────────────────────────────────
_BEST_PV_BY_ENV: Dict[Tuple[str, str, float], float] = {}
_BEST_SEL_BY_ENV: Dict[Tuple[str, str, float], Dict[Tuple[str, int], int]] = {}

# ─────────────────────────────────────────────────────────────────────
# Objective construction
# ─────────────────────────────────────────────────────────────────────
def _set_weighted_objective(
    M: CoptSpendMatchMO,
    expr_primary_pv: co.LinExpr,
    *,
    tag: str,
) -> None:
    """
    Objective:
        Minimise  BACKLOG_WEIGHT * ∑ backlog[t]
                + ∑ (Weight_i * Excess_Tier_i[t])  <-- Piecewise Penalty
                - PV_WEIGHT      * PV
    """

    obj_backlog = BACKLOG_WEIGHT * M.annual_backlog_sum
    obj_excess = M.tier_penalties_expr
    obj_pv = expr_primary_pv * PV_WEIGHT

    combined_obj = obj_backlog + obj_excess - obj_pv
    M.m.setObjective(combined_obj, COPT.MINIMIZE)

    _log(
        f"[{tag}] Objective: MIN("
        f" {BACKLOG_WEIGHT:.1f} * Backlog"
        f" + PiecewiseExcessPenalty"
        f" - {PV_WEIGHT:.1e} * PV )"
    )

@dataclass
class Incumbent:
    sel: Dict[Tuple[str, int], int]
    pv: float
    max_net_balance: float      # maximum net balance over the horizon (M)
    annual_backlog_sum: float   # total backlog over the horizon (M)
    val_by_id: Dict[int, float]
    model: CoptSpendMatchMO
    gap: Optional[float]

def build_total_model(
    variants: Dict[str, dict],
    allowed: Dict[str, List[int]],
    funding_target_S: np.ndarray,
    Tfine: int,
    taper_start_idx: int,
    coeff_total_fine_int: Dict[Tuple[str, int], int],
    total_floor_target: Optional[float],
    is_whitelist_model: bool,
    *,
    relax_binaries: bool = False,
) -> CoptSpendMatchMO:
    M = CoptSpendMatchMO(
        variants,
        allowed,
        funding_target_S,
        Tfine,
        taper_start_idx,
        spend_by_year={v: variants[v]["spend"] for v in variants},
        max_starts_per_year=MAX_STARTS_PER_FY,
        is_whitelist_model=is_whitelist_model,
        name="MO_SPEND_MATCH",
        relax_binaries=relax_binaries,
    )

    expr_tot_pv = M.obj_expr(coeff_total_fine_int, cache_key="TOT_PV")

    if total_floor_target is not None:
        floor_target = total_floor_target - max(
            MONO_ABS_EPS, MONO_REL_EPS * abs(total_floor_target)
        )
        M.add_floor(expr_tot_pv, floor_target, name="mono_total_floor")

    _set_weighted_objective(M, expr_tot_pv, tag="TOTAL")
    return M

def eval_incumbent(
    M: CoptSpendMatchMO,
    val_by_id: Dict[int, float],
    coeff_total_fine: Dict[Tuple[str, int], float],
) -> Incumbent:
    sel = selection_from_values(M.x, val_by_id)
    pv = pv_from_selection(coeff_total_fine, sel)

    net_vals_M = [_val(val_by_id, M.net[t]) / SPEND_SCALE for t in range(M.Tn)]
    max_net_balance = max(net_vals_M) if net_vals_M else 0.0

    backlog_vals_M = [_val(val_by_id, M.backlog[t]) / SPEND_SCALE for t in range(M.Tn)]
    annual_backlog_sum = sum(backlog_vals_M)

    gap = _best_gap(M.m)
    return Incumbent(
        sel=sel,
        pv=pv,
        max_net_balance=max_net_balance,
        annual_backlog_sum=annual_backlog_sum,
        val_by_id=val_by_id,
        model=M,
        gap=gap,
    )

# ─────────────────────────────────────────────────────────────────────
# Orchestrator (Piecewise)
# ─────────────────────────────────────────────────────────────────────
def orchestrate_total(
    variants: Dict[str, dict],
    allowed_fine: Dict[str, List[int]],
    funding_target_S: np.ndarray,
    Tfine: int,
    taper_start_idx: int,
    coeff_total_fine: Dict[Tuple[str, int], float],
    coeff_total_fine_int: Dict[Tuple[str, int], int],
    total_floor_target: Optional[float],
    is_whitelist_model: bool,
    *,
    sel_seed: Optional[Dict[Tuple[str, int], int]],
    rel_gap_target: float,
    time_budget_s: float,
) -> Tuple[Optional[Incumbent], Optional[float]]:
    _log(
        f"[orchestrate] Building model with FIXED FUNDING + PIECEWISE SOFT CAP. "
        f"Time budget: {time_budget_s:.0f}s, target gap={rel_gap_target:.4f}"
    )

    M = build_total_model(
        variants,
        allowed_fine,
        funding_target_S,
        Tfine,
        taper_start_idx,
        coeff_total_fine_int,
        total_floor_target,
        is_whitelist_model,
    )

    if sel_seed:
        M.add_hint_from_starts(sel_seed)

    res = solve_model(
        M.m,
        stage="MAIN_SOLVE",
        base_time=time_budget_s,
        rel_gap=rel_gap_target,
    )

    if not res.has_inc:
        _log("[orchestrate] Solve finished with no incumbent solution.")
        return None, res.gap

    # Extract solution values from the model
    valmap = extract_solution_values(M)
    if not valmap:
        _log("[orchestrate] Solve failed to extract variable values.")
        return None, res.gap

    best_inc = eval_incumbent(M, valmap, coeff_total_fine)
    _log(
        "[orchestrate] Solve complete. "
        f"BacklogSum: {best_inc.annual_backlog_sum:,.1f} M, "
        f"MaxNetBalance: {best_inc.max_net_balance:,.1f} M, "
        f"PV: {best_inc.pv:,.1f}, "
        f"Gap: {best_inc.gap or float('nan'):.4%}"
    )
    return best_inc, best_inc.gap

# ─────────────────────────────────────────────────────────────────────
# DIMENSION run (weighted)
# ─────────────────────────────────────────────────────────────────────
TOTAL_PV_GUARD_PCT = 85.0

def weights_for_dimension(
    primary_dim: str,
    variants: Dict[str, dict],
    kernels_by_dim: Dict[str, Dict[str, List[float]]],
    r: float,
) -> Dict[str, float]:
    alpha = 1.0
    beta = 0.5
    wmin = 1.0
    wmax = 2.0
    eps = 1e-9
    dims = list(kernels_by_dim.keys())
    if primary_dim not in kernels_by_dim:
        return {v: 1.0 for v in variants}

    def pv_start0(ker: List[float], r_: float) -> float:
        pv = 0.0
        denom = 1.0
        g = 1.0 + r_
        for f in ker:
            pv += float(f) / denom
            denom *= g
        return pv

    pv0_primary = {
        v: pv_start0(kernels_by_dim[primary_dim].get(v, []), r) for v in variants
    }
    arr = np.array(list(pv0_primary.values()))
    order = np.argsort(arr)
    ranks = np.empty_like(arr, dtype=float)
    ranks[order] = np.arange(1, len(arr) + 1, dtype=float)
    pct = {
        k: float(max(ranks[i] / float(len(arr)), eps))
        for i, k in enumerate(pv0_primary.keys())
    }

    others = [d for d in dims if d != primary_dim and d.lower() != "total"]
    pv0_other_mean: Dict[str, float] = {}
    for v in variants:
        vals = [
            pv_start0(kernels_by_dim.get(d, {}).get(v, []), r)
            for d in others
            if kernels_by_dim.get(d, {}).get(v)
        ]
        pv0_other_mean[v] = float(np.mean(vals)) if vals else 0.0

    ratio: Dict[str, float] = {}
    for v in variants:
        denom = max(pv0_other_mean[v], eps)
        if denom > 0:
            ratio[v] = pv0_primary[v] / denom
        else:
            ratio[v] = wmax if pv0_primary[v] > 0 else 1.0

    w: Dict[str, float] = {}
    for v in variants:
        wv = (max(ratio[v], eps)) ** alpha * (max(pct[v], eps)) ** beta
        w[v] = float(wv)
    m = float(np.mean(list(w.values()))) if w else 1.0
    if m > eps:
        w = {k: v / m for k, v in w.items()}

    return {k: float(min(max(v, wmin), wmax)) for k, v in w.items()}

def coeff_map_for_dim_fine_weighted(
    dim: str,
    variants: Dict[str, dict],
    kernels_by_dim: Dict[str, Dict[str, List[float]]],
    allowed_fine: Dict[str, List[int]],
    Tfine: int,
    disc_vec: np.ndarray,
) -> Tuple[Dict[Tuple[str, int], float], Dict[Tuple[str, int], int]]:
    w = weights_for_dimension(dim, variants, kernels_by_dim, BENEFIT_DISCOUNT_RATE)
    base = coeff_map_for_dim_fine(
        variants, kernels_by_dim.get(dim, {}), allowed_fine, Tfine, disc_vec
    )
    if not base:
        return {}, {}
    weighted = {
        (v, s): base.get((v, s), 0.0) * float(w.get(v, 1.0)) for (v, s) in base
    }
    return weighted, coeff_int(weighted)

def weighted_coeff_maps_for_dimension(
    dim: str,
    variants: Dict[str, dict],
    kernels_by_dim: Dict[str, Dict[str, List[float]]],
    allowed_fine: Dict[str, List[int]],
    Tfine: int,
    disc_vec: np.ndarray,
) -> Tuple[Dict[Tuple[str, int], float], Dict[Tuple[str, int], int]]:
    return coeff_map_for_dim_fine_weighted(
        dim, variants, kernels_by_dim, allowed_fine, Tfine, disc_vec
    )

def run_dimensions_for_env(
    ct: str,
    sc_key: str,
    sc_sheet: str,
    sur_key: str,
    plus_M: float,
    *,
    projects: Dict[str, dict],
    variants: Dict[str, dict],
    costs_input_df: pd.DataFrame,
    ben_kernel_df: pd.DataFrame,
    kernels_by_dim: Dict[str, Dict[str, List[float]]],
    Tfine: int,
    taper_start_idx: int,
    funding_target_M: np.ndarray,
    funding_target_S: np.ndarray,
    allowed_fine: Dict[str, List[int]],
    disc_vec: np.ndarray,
    coeff_total_fine_int: Dict[Tuple[str, int], int],
    total_best_sel: Dict[Tuple[str, int], int],
    total_best_pv: float,
    is_whitelist_model: bool,
    tot_model_for_fallback: Optional[CoptSpendMatchMO],
    tot_valmap_for_fallback: Optional[Dict[int, float]],
    prev_dim_floors: Optional[Dict[str, float]] = None,
    dims_filter: Optional[Iterable[str]] = None,
) -> Dict[str, float]:
    prev_dim_floors = dict(prev_dim_floors or {})
    dims_all = [d for d in kernels_by_dim.keys() if d.lower() != "total"]
    if dims_filter is not None:
        dims = [d for d in dims_all if d in set(dims_filter)]
    else:
        dims = dims_all

    if not dims:
        _log(" [DIM] No weighted-dimension runs requested.")
        return prev_dim_floors

    total_floor = float(total_best_pv) * (TOTAL_PV_GUARD_PCT / 100.0)

    for dim in dims:
        tag_dim = (
            f"{ct.replace(' ','').replace('-', '')}_{sc_key}_{sur_key}_pm{int(plus_M)}_"
            f"{dim_short(dim)}"
        )
        _log(f" [DIM] {dim} (weighted) with Total-guard ≥ {total_floor:,.6f}")

        coeff_dim_w_f, coeff_dim_w_int = weighted_coeff_maps_for_dimension(
            dim, variants, kernels_by_dim, allowed_fine, Tfine, disc_vec
        )

        if not coeff_dim_w_int:
            if tot_model_for_fallback is not None:
                dump_pickle_full(
                    tot_model_for_fallback,
                    tag_dim,
                    projects=projects,
                    variants=variants,
                    costs_input_df=costs_input_df,
                    ben_kernel_df=ben_kernel_df,
                    kernels_by_dim=kernels_by_dim,
                    benefit_rate=BENEFIT_DISCOUNT_RATE,
                    scenario_name=sc_key,
                    primary_dim=f"{dim} [WEIGHTED]",
                    Tfine=Tfine,
                    funding_target_M=funding_target_M,
                    sel_override=total_best_sel,
                    val_override=tot_valmap_for_fallback,
                    gap_override=_best_gap(tot_model_for_fallback.m),
                    status_override="OK_SUBOPT_NOCOEFF",
                    extra_diag={
                        "note": "dimension has zero coefficients; reported Total solution"
                    },
                )
            prev_dim_floors[dim] = pv_from_selection(coeff_dim_w_f, total_best_sel)
            continue

        Md = CoptSpendMatchMO(
            variants,
            allowed_fine,
            funding_target_S,
            Tfine,
            taper_start_idx,
            spend_by_year={v: variants[v]["spend"] for v in variants},
            max_starts_per_year=MAX_STARTS_PER_FY,
            is_whitelist_model=is_whitelist_model,
            name=f"MO_DIM_{dim_short(dim)}",
        )
        Md.add_hint_from_starts(total_best_sel)

        expr_dim_pv = Md.obj_expr(
            coeff_dim_w_int, cache_key=f"DIM_{dim_short(dim)}"
        )

        # Total PV guard
        expr_tot_guard = Md.obj_expr(
            coeff_total_fine_int, cache_key="TOT_GUARD"
        )
        Md.add_floor(
            expr_tot_guard,
            total_floor - max(MONO_ABS_EPS, MONO_REL_EPS * abs(total_floor)),
            name="dim_total_floor",
        )

        _set_weighted_objective(
            Md, expr_dim_pv, tag=f"DIM_{dim_short(dim)}"
        )

        base_time = EFFORT[OPTIMISATION_PROFILE]["MO"]
        rel_gap = EFFORT[OPTIMISATION_PROFILE]["REL_GAP"]
        res = solve_model(
            Md.m,
            stage=f"DIM[{dim_short(dim)}]",
            base_time=base_time,
            rel_gap=rel_gap,
        )

        if _has_incumbent(Md.m):
            valmap = extract_solution_values(Md)
            if valmap:
                assert_and_log_invariants(
                    Md, funding_target_M, valmap, label=tag_dim
                )
                dump_pickle_full(
                    Md,
                    tag_dim,
                    projects=projects,
                    variants=variants,
                    costs_input_df=costs_input_df,
                    ben_kernel_df=ben_kernel_df,
                    kernels_by_dim=kernels_by_dim,
                    benefit_rate=BENEFIT_DISCOUNT_RATE,
                    scenario_name=sc_key,
                    primary_dim=f"{dim} [WEIGHTED]",
                    Tfine=Tfine,
                    funding_target_M=funding_target_M,
                    sel_override=selection_from_values(Md.x, valmap),
                    val_override=valmap,
                    gap_override=res.gap,
                )
            else:
                _log(f"[DIM] {dim} solve OK but value extraction failed.")
                if tot_model_for_fallback is not None:
                    dump_pickle_full(
                        tot_model_for_fallback,
                        tag_dim,
                        projects=projects,
                        variants=variants,
                        costs_input_df=costs_input_df,
                        ben_kernel_df=ben_kernel_df,
                        kernels_by_dim=kernels_by_dim,
                        benefit_rate=BENEFIT_DISCOUNT_RATE,
                        scenario_name=sc_key,
                        primary_dim=f"{dim} [WEIGHTED]",
                        Tfine=Tfine,
                        funding_target_M=funding_target_M,
                        sel_override=total_best_sel,
                        val_override=tot_valmap_for_fallback,
                        gap_override=_best_gap(tot_model_for_fallback.m),
                        status_override="OK_SUBOPT_VALFAIL",
                        extra_diag={
                            "note": "dimension solver ok but val extract failed; "
                            "reported Total solution"
                        },
                    )
        else:
            _log(f"[DIM] {dim} solve failed to find incumbent.")
            if tot_model_for_fallback is not None:
                dump_pickle_full(
                    tot_model_for_fallback,
                    tag_dim,
                    projects=projects,
                    variants=variants,
                    costs_input_df=costs_input_df,
                    ben_kernel_df=ben_kernel_df,
                    kernels_by_dim=kernels_by_dim,
                    benefit_rate=BENEFIT_DISCOUNT_RATE,
                    scenario_name=sc_key,
                    primary_dim=f"{dim} [WEIGHTED]",
                    Tfine=Tfine,
                    funding_target_M=funding_target_M,
                    sel_override=total_best_sel,
                    val_override=tot_valmap_for_fallback,
                    gap_override=_best_gap(tot_model_for_fallback.m),
                    status_override="OK_SUBOPT_NOSOLVE",
                    extra_diag={
                        "note": "dimension solver yielded no incumbent; "
                        "reported Total solution"
                    },
                )
    return prev_dim_floors

# ─────────────────────────────────────────────────────────────────────
# Helpers for dimension names
# ─────────────────────────────────────────────────────────────────────
_DIM_SHORT: Dict[str, str] = {
    "Total": "TOT",
    "Healthy and safe people": "HSP",
    "Inclusive Access": "INC",
    "Environmental Sustainability": "ENV",
    "Economic Prosperity": "ECO",
    "Resilience and Security": "RES",
}

def dim_short(dim: str) -> str:
    if dim in _DIM_SHORT:
        return _DIM_SHORT[dim]
    toks = re.findall(r"[A-Za-z0-9]+", dim or "")
    return ("".join(t[:3] for t in toks)[:8] or "DIM").upper()

# ─────────────────────────────────────────────────────────────────────
# Run combo
# ─────────────────────────────────────────────────────────────────────
def get_funding_envelope(
    Tfine: int,
    base_M: float,
    taper_start_from_end: int,
) -> Tuple[np.ndarray, int]:
    """
    Build the envelope *capacity* path (M) and return:
        funding_M[t]  – max available funding in year t (constant base_M)
        taper_start_idx – dummy index (no taper; set to Tfine)
    """
    funding_M = np.full(Tfine, float(base_M), dtype=float)
    taper_start_idx = Tfine  # no taper region; kept for compatibility
    return funding_M, taper_start_idx

def run_combo(
    ct: str,
    sc_key: str,
    sc_sheet: str,
    sur_key: str,
    baseline_surplus_M: float,
    plus_M: float,
    prev_dim_floors: Optional[Dict[str, float]] = None,
    prev_best_sel: Optional[Dict[Tuple[str, int], int]] = None,
):
    projects_all, variants_all, costs_input_df = load_costs(ct)

    variants, forced_exact, is_whitelist_model = apply_forced_rules(
        variants_all, FORCED_START
    )
    projects = {p: projects_all[p] for p in variants.keys() if p in projects_all}

    # --- PROCESS MIN START YEARS ---
    min_start_norm = {norm(k): v for k, v in (MIN_START_YEAR or {}).items()}
    min_start_exact: Dict[str, int] = {}
    for v in variants:
        n = norm(v)
        if n in min_start_norm:
            min_start_exact[v] = min_start_norm[n]
    # -------------------------------

    benef_df, ben_kernel_df = load_benefits(sc_sheet)
    dims_order, kernels_by_dim = map_benefit_kernels(benef_df, variants)
    total_key = "Total"

    include_norm = {norm(k): bool(v) for k, v in (DIMENSION_INCLUSIONS or {}).items()}
    dims_to_run = [
        d
        for d in kernels_by_dim.keys()
        if d.lower() != "total" and include_norm.get(norm(d), False)
    ]

    full_envelope_M = float(baseline_surplus_M) + float(plus_M)

    # Fixed horizon across buffers so that feasibility is comparable and
    # the monotone PV guard makes sense.
    Tfine = TFIXED

    tot_cost = float(sum(sum(meta["spend"]) for meta in variants.values()))
    _log(
        f" Fixed Horizon: TotCost={tot_cost:,.0f}M, Env={full_envelope_M:,.0f}M/yr "
        f"-> Years={Tfine}"
    )
    _log(f" PV window: {START_FY}→{START_FY + Tfine - 1} (Tfine={Tfine})")

    funding_target_M, taper_start_idx = get_funding_envelope(
        Tfine,
        full_envelope_M,
        taper_start_from_end=TAPER_YEARS,
    )
    funding_target_S = np.array(
        [iround(m, SPEND_SCALE) for m in funding_target_M],
        dtype=float,
    )
    _log(
        f" Envelope capacity: up to {full_envelope_M:,.0f} M per year "
        f"(chosen as needed)."
    )

    tag_tot = (
        f"{ct.replace(' ','').replace('-', '')}_{sc_key}_{sur_key}_pm{int(plus_M)}_"
        f"{_DIM_SHORT['Total']}"
    )

    total_funding_M = float(np.sum(funding_target_M))

    if is_whitelist_model and (tot_cost > total_funding_M + 1e-9):
        msg = (
            f"[WHITELIST] Total project spend {tot_cost:,.1f} M exceeds *total* "
            f"funding capacity {total_funding_M:,.1f} M. Scenario infeasible."
        )
        print(" ×", msg)
        fn = CACHE / f"{(PKL_PREFIX or '')}{tag_tot}_noSol.pkl"
        pkl_save(
            fn,
            {
                "status": "NoSolve",
                "reason": "mass-balance infeasible (whitelist)",
                "detail": msg,
                "objective": "Total",
                "created_at": _now_stamp(),
                "gap": None,
                "gap_pct": None,
            },
        )
        return None, prev_dim_floors, prev_best_sel

    allowed_fine = allowed_starts_fine(
        variants, forced_exact, min_start_exact, Tfine
    )
    missing = [p for p in variants if not allowed_fine.get(p)]
    if missing:
        fn = CACHE / f"{(PKL_PREFIX or '')}{tag_tot}_noSol.pkl"
        pkl_save(
            fn,
            {
                "status": "NoSolve",
                "reason": f"no allowed starts for {missing[:5]}",
                "diagnostic": {"phase": "screen", "Tfine": Tfine},
                "objective": "Total",
                "created_at": _now_stamp(),
                "gap": None,
                "gap_pct": None,
            },
        )
        return None, prev_dim_floors, prev_best_sel

    disc_vec = np.array(
        [(1.0 + BENEFIT_DISCOUNT_RATE) ** t for t in range(Tfine)],
        dtype=float,
    )
    coeff_total_fine = coeff_map_for_dim_fine(
        variants, kernels_by_dim[total_key], allowed_fine, Tfine, disc_vec
    )
    coeff_total_fine_int = coeff_int(coeff_total_fine)

    sel_hint = greedy_warm_start(
        variants,
        allowed_fine,
        Tfine,
        funding_target_M,
        MAX_STARTS_PER_FY,
        forced_exact,
        coeff_total_fine,
        reuse_sel=prev_best_sel,
    )

    prev_env = max(
        [
            e
            for (ctk, sck, e) in _BEST_PV_BY_ENV.keys()
            if ctk == ct and sck == sc_key and e < full_envelope_M
        ],
        default=None,
    )
    total_floor_target = None
    if ENFORCE_MONOTONE_PV_ACROSS_BUFFERS and prev_env is not None:
        prev_best = _BEST_PV_BY_ENV[(ct, sc_key, prev_env)]
        total_floor_target = prev_best

    prof_effort = EFFORT[OPTIMISATION_PROFILE]
    best_inc, final_gap = orchestrate_total(
        variants,
        allowed_fine,
        funding_target_S,
        Tfine,
        taper_start_idx,
        coeff_total_fine,
        coeff_total_fine_int,
        total_floor_target,
        is_whitelist_model,
        sel_seed=sel_hint,
        rel_gap_target=prof_effort["REL_GAP"],
        time_budget_s=float(prof_effort["MO"]),
    )

    if best_inc is None or best_inc.model is None:
        fn = CACHE / f"{(PKL_PREFIX or '')}{tag_tot}_noSol.pkl"
        pkl_save(
            fn,
            {
                "status": "NoSolve",
                "reason": "no incumbent with piecewise cap",
                "objective": "Total",
                "created_at": _now_stamp(),
                "diagnostic": {
                    "note": "No feasible solution found.",
                },
                "gap": None,
                "gap_pct": None,
            },
        )
        return None, prev_dim_floors, prev_best_sel

    extra_diag = {
        "orchestrated_gap": f"{(final_gap if final_gap is not None else float('nan')):.4%}",
        "net_cap_tiers": str(PIECEWISE_CAP_TIERS),
    }

    assert_and_log_invariants(
        best_inc.model, funding_target_M, best_inc.val_by_id, label=tag_tot
    )

    dump_pickle_full(
        best_inc.model,
        tag_tot,
        projects=projects,
        variants=variants,
        costs_input_df=costs_input_df,
        ben_kernel_df=ben_kernel_df,
        kernels_by_dim=kernels_by_dim,
        benefit_rate=BENEFIT_DISCOUNT_RATE,
        scenario_name=sc_key,
        primary_dim="Total",
        Tfine=Tfine,
        funding_target_M=funding_target_M,
        sel_override=best_inc.sel,
        val_override=best_inc.val_by_id,
        gap_override=best_inc.gap,
        extra_diag=extra_diag,
    )

    best_sel_for_next = best_inc.sel
    valmap_for_fallback = best_inc.val_by_id

    _BEST_PV_BY_ENV[(ct, sc_key, full_envelope_M)] = pv_from_selection(
        coeff_total_fine, best_sel_for_next
    )
    if best_sel_for_next:
        _BEST_SEL_BY_ENV[(ct, sc_key, full_envelope_M)] = best_sel_for_next

    # Weighted-dimension runs
    include_norm = {norm(k): bool(v) for k, v in (DIMENSION_INCLUSIONS or {}).items()}
    dims_to_run = [
        d
        for d in kernels_by_dim.keys()
        if d.lower() != "total" and include_norm.get(norm(d), False)
    ]

    disc_vec = np.array(
        [(1.0 + BENEFIT_DISCOUNT_RATE) ** t for t in range(Tfine)],
        dtype=float,
    )

    prev_dims = run_dimensions_for_env(
        ct,
        sc_key,
        sc_sheet,
        sur_key,
        plus_M,
        projects=projects,
        variants=variants,
        costs_input_df=costs_input_df,
        ben_kernel_df=ben_kernel_df,
        kernels_by_dim=kernels_by_dim,
        Tfine=Tfine,
        taper_start_idx=taper_start_idx,
        funding_target_M=funding_target_M,
        funding_target_S=funding_target_S,
        allowed_fine=allowed_fine,
        disc_vec=disc_vec,
        coeff_total_fine_int=coeff_total_fine_int,
        total_best_sel=best_sel_for_next,
        total_best_pv=pv_from_selection(
            coeff_total_fine, best_sel_for_next
        ),
        is_whitelist_model=is_whitelist_model,
        tot_model_for_fallback=best_inc.model,
        tot_valmap_for_fallback=valmap_for_fallback,
        prev_dim_floors=prev_dim_floors,
        dims_filter=dims_to_run,
    )

    return (
        pv_from_selection(coeff_total_fine, best_sel_for_next),
        prev_dims,
        best_sel_for_next,
    )

# ─────────────────────────────────────────────────────────────────────
# Driver
# ─────────────────────────────────────────────────────────────────────
def main() -> None:
    random.seed(SOLVER_SEED_DEFAULT)
    np.random.seed(SOLVER_SEED_DEFAULT)

    global _BEST_PV_BY_ENV, _BEST_SEL_BY_ENV
    _BEST_PV_BY_ENV.clear()
    _BEST_SEL_BY_ENV.clear()

    print(
        f"CFG: START_FY={START_FY} FINAL_YEAR={FINAL_YEAR} "
        f"PV_WINDOW={TFIXED}y MAX_STARTS/FY={MAX_STARTS_PER_FY}"
    )
    eff = EFFORT[OPTIMISATION_PROFILE]
    print(
        f"Effort: {OPTIMISATION_PROFILE} → "
        f"{{'MO': {eff['MO']}, 'REL_GAP': {eff['REL_GAP']}}}"
    )
    print("NEW MODEL (v94.14 - Hybrid):")
    print("  - FIXED FUNDING: funding[t] == Envelope.")
    print("  - DIVIDEND (Restricted): Only allowed at end-of-life (y[t+1]=0).")
    print("  - PIECEWISE SOFT CAP: Tiers 12%/15%/20%.")
    print("  - ALL PROJECTS: sum(x) == 1.0 (Mandatory Selection).")
    print("  - FIXED PV HORIZON: Tfine = TFIXED.")
    print("  - PROFILE-DEPENDENT SOLVE:")
    print("      fast/balanced → single-phase (v94.11 style)")
    print("      thorough      → 2-phase (Probe + Verify + Victory Lap)")
    print("      ultra         → 3-phase (Probe + Verify + Victory Lap + Grind)\n")

    if FORCED_START:
        print(f"FORCED_START: {len(FORCED_START)} rules are active.")
    if MIN_START_YEAR:
        print(f"MIN_START_YEAR: {len(MIN_START_YEAR)} constraint rules are active.")

    dims_incl_str = ", ".join(
        [k for k, v in DIMENSION_INCLUSIONS.items() if k.lower() != "total" and v]
    )
    print(
        f"Dimension inclusions (non-Total): "
        f"{dims_incl_str if dims_incl_str else 'None'}"
    )
    print(f"RUN_ID: {RUN_ID}\n")

    for ct in COST_TYPES_RUN:
        print(f"\n=== COST: {ct} ===")
        for sc_key, sc_sheet in BENEFIT_SCENARIOS.items():
            print(f">> Scenario {sc_key} (sheet {sc_sheet})")
            prev_sel: Optional[Dict[Tuple[str, int], int]] = None
            prev_dims: Dict[str, float] = {}
            envs_sorted = sorted(
                SURPLUS_OPTIONS_M.items(), key=lambda kv: kv[1]
            )

            for sur_key, baseM in envs_sorted:
                print(f" -- Base Funding {sur_key} = {baseM:.0f} M p.a. --")
                for plus in sorted(PLUSMINUS_LEVELS_M):
                    print(f" Running... FULL={baseM+plus:,.0f} M")
                    out = run_combo(
                        ct,
                        sc_key,
                        sc_sheet,
                        sur_key,
                        baseM,
                        plus,
                        prev_dim_floors=prev_dims,
                        prev_best_sel=prev_sel,
                    )
                    if out is not None:
                        tot_pv, prev_dims, prev_sel = out

    print("\n✓ Done. Pickles are under:", CACHE)

if __name__ == "__main__":
    t0 = time.time()
    try:
        main()
    finally:
        print(f"\nRun-time: {time.time()-t0:.1f}s → {CACHE}")
